# Introduction/Business Problem

# Data

### Origin of the Data

### Preparation of the Data Gathering

In [1]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import folium

print('Libraries imported.')

Libraries imported.


Foursquare API Credentials Definitions

In [2]:
CLIENT_ID = 'TYN2NSLD2H3O33G1GKM3KSWIXTN40P0NNWHSBKCTKCPTFNBV' # your Foursquare ID
CLIENT_SECRET = '3D0B3UI55ZIROTX3LC5JFI3XCE5ZHJXJ0GSQ05V3YINQQUCL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TYN2NSLD2H3O33G1GKM3KSWIXTN40P0NNWHSBKCTKCPTFNBV
CLIENT_SECRET:3D0B3UI55ZIROTX3LC5JFI3XCE5ZHJXJ0GSQ05V3YINQQUCL
